# Interacting with Coq

The `Pytanque` class implements a lightweight client that can communicate with the Rocq prover via the `pet-server`.

In [1]:
from pytanque import Pytanque

First you need to start the server.
Instructions can be found in the coq-lsp repo (https://github.com/ejgallego/coq-lsp/tree/main/petanque)

Then, you can create a client and connect it to the same address and port.
You need to indicate the location of the project (directory containing the `_CoqProject` file, or root directory for the `.v` files).

In [2]:
pet = Pytanque("127.0.0.1", 8765)
pet.connect()
pet.init(root=".")


Petanque relies on regular coq files.
To prove an existing theorem, just indicate the location of the `.v` file and the name of the theorem.

In [3]:
pet.start(file="./foo.v", thm="addnC")

Internally the state is a simple `id`.
To retrieve the state of the prover, you can use the `goals` method.

In [4]:
g = pet.current_state()
print(f"Internal pytanque state: {g}")

goals = pet.goals() # Retrieve actual state
print(f"Prover goals: {goals}\n")
goals.pp() # Pretty print the current goals

Internal pytanque state: 13
Prover goals: GoalsResponse(goals=[Goal(info={'evar': ['Ser_Evar', 12], 'name': None}, hyps=[GoalHyp(names=['n', 'm'], ty='nat', def_=None)], ty='n + m = m + n')])

n, m  : nat
-----------------------------
n + m = m + n





To execute a tactic, use the `run_tac` method.

In [5]:
pet.run_tac("induction n.")
goals = pet.goals()
goals.pp()

m  : nat
-----------------------------
0 + m = m + 0



n, m  : nat
IHn  : n + m = m + n
-----------------------------
S n + m = m + S n





You can always undo a tactic with the `backtrack` method, or restart the entire proof of the current theorem with `reset`.

In [6]:
(st, tac) = pet.backtrack()
print(f"Undo {tac}, back to {st}")

Undo induction n., back to 14


In [7]:
pet.reset()
pet.run_tac("by elim: n => //= ? ->.")

ProofFinished(value=16)